### Dyer-Lashof-Cohen basis

In [ ]:
from itertools import combinations, combinations_with_replacement, permutations, combinations, product, chain, tee
from sympy.utilities.iterables import multiset_permutations
from clesto import *

def symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: 1}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

def sign_symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    sign = {(1,2): 1, (1,3): -1, (2,1): -1,
            (2,3): 1, (3,1): 1, (3,2): -1}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: sign[k[:2]]}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

# def sign_permutation(permutation):
#     perm = [i - 1 for i in permutation]
#     visited = [False] * len(perm)
#     parity = 0
#     for i in perm:
#         visited[i] = True
#         cycle_length = 0
#         while not visited[perm[i]]:
#             visited[perm[i]] = True
#             i = perm[i]
#             cycle_length += 1
#         parity += cycle_length % 2

#     return (-1)**parity 

# def symmetric_orbit_barratt_eccles(be, sign_representation=False):
#     rep = BarrattEccles_element(torsion=be.torsion)
#     for k, v in be.items():
#         inverse = tuple(k[-1].index(i+1)+1 for i in range(len(k[0])))
#         perm = SymmetricModule_element({inverse: 1}, torsion=be.torsion)
#         new = perm * BarrattEccles_element({k: v}, torsion=be.torsion)
#         if sign_representation:
#             new = sign_permutation(k[-1]) * new
#         rep += new
#     return rep

def symmetric_orbit_barratt_eccles(be, sign_representation=False):
    inverses = {(1,2,3): (1,2,3), (2,3,1): (3,1,2), (3,1,2): (2,3,1), 
                (1,3,2): (1,3,2), (3,2,1): (3,2,1), (2,1,3): (2,1,3)}
    signs = {(1,2,3): 1, (2,3,1): 1, (3,1,2): 1, 
             (1,3,2): -1, (3,2,1): -1, (2,1,3): -1}
    
    answer = BarrattEccles_element(torsion=be.torsion)
    for k, v in be.items():
        inverse = k[-1]
        perm = SymmetricModule_element({inverse: 1}, torsion=be.torsion)
        new = perm * BarrattEccles_element({k: v}, torsion=be.torsion)
        if sign_representation:
            new = signs[k[-1]] * new
        answer += new
        
    return answer

def extra_reduce(surj, r=3):
    return Surjection_element({k: v for k, v in surj.items() if set(k) == set(range(1, r+1))})

def cyclic_orbit(surj):
    pass

def get_symmetric_basis():
    basis = set()
    for x in combinations_with_replacement((1,2,3), 5):
        for y in permutations(x):
            if set(y) == {1,2,3} and Surjection_element({y:1}):
                basis.update(symmetric_orbit(Surjection_element({y:1}, torsion=3)))
    return basis

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def get_basis(r, n, torsion='free'):
    basis = []
    surjections = (chain.from_iterable(multiset_permutations(y) 
        for y in combinations_with_replacement(range(1, r+1), r+n)))
    for s in surjections:
        if all([i != j for i, j in pairwise(s)]) and set(s) == set(range(1, r+1)):
#             yield Surjection_element({tuple(s):1}, torsion=torsion)
            yield tuple(s)
        
def phi(i, j, permutations):
    '''the restriction to a pair of integers a la Berger, i.e pull back to the arity 2 part'''
    def _phi(i, j, permutation):
        try:
            a, b = permutation.index(i), permutation.index(j)

        except ValueError:
            print(i, j, permutation)

        if a < b:
            return (1, 2)
        else:
            return (2, 1)
        
    return tuple(_phi(i, j, p) for p in permutations)

def is_nondegenerate(spx):
    return all(spx[i] != spx[i + 1] for i in range(len(spx) - 1))

def phi(i, j, barratt_eccles_element):
    
    def _phi(i, j, permutation):
        a, b = permutation.index(i), permutation.index(j)
        if a < b:
            return (1, 2)
        else:
            return (2, 1)
        
    if isinstance(barratt_eccles_element, tuple):
        return tuple(_phi(i, j, p) for p in barratt_eccles_element)
    
    torsion = barratt_eccles_element.torsion
    answer = BarrattEccles_element(torsion=torsion)
    for k, v in barratt_eccles_element.items():
        new_simplex = []
        new_simplex = phi(i, j, k)
        new_summand = BarrattEccles_element({new_simplex: v}, torsion=torsion)
        answer += new_summand

def complexity(barratt_eccles_element):
    "Not finished yet"
    
    def _complexity(spx):
        complexities = [0]
        for i, j in combinations(sorted(spx[0]), 2):
            cpxty = len([p for p, q in pairwise(phi(i, j, spx)) if p != q])
            complexities.append(cpxty)
        return max(complexities)
    
    if isinstance(barratt_eccles_element, tuple):
        return _complexity(barratt_eccles_element)

## Wasteful approach

In [ ]:
import itertools 

# computes all simplices in Barratt-Eccles operad with

r = 3 # arity
n = 2 # dimension
c = 1 # complexity i.e. in E_{c+1}

permutations = set(itertools.permutations(range(1, r+1), r))
ordered_simplices = combinations_with_replacement(permutations, n+1)
simplices = set()
for y in ordered_simplices:
    simplices |= set(itertools.permutations(y))

# removes degenerate simplices
simplices = tuple(spx for spx in simplices if is_nondegenerate(spx))

# remove simplices of high complexity
simplices = tuple(spx for spx in simplices if complexity(spx) <= c)

# remove simplices not in the top dim cell
cell = []
for spx in simplices:
    in_cell = True
    for i, j in combinations(sorted(spx[0]), 2):
        if (phi(i, j, spx)[-1] == (2,1) and
            len([p for p, q in pairwise(phi(i, j, spx)) if p != q]) == c):
            in_cell = False
    if in_cell:
        cell.append(spx)

for spx in cell:
    weights = []
    print(spx)
    print(BarrattEccles_element({spx: 1}, torsion=3).table_reduction())
    print('---')

In [ ]:
inverse = {(1, 3, 2): (1, 3, 2), (3, 2, 1): (3, 2, 1), (2, 1, 3): (2, 1, 3),
           (1, 2, 3): (1, 2, 3), (3, 1, 2):(2, 3, 1), (2, 3, 1):(3, 1, 2)}
for spx in simplices:
    print(tuple(act(perm, inverse[spx[-1]]) for perm in spx))

In [ ]:
from itertools import combinations, product, permutations, combinations_with_replacement, accumulate
from sympy import binomial

def act (perm1, perm2):
    return tuple(perm1[i-1] for i in perm2)

r = 3
n = 2

transpositions = []
identity = tuple(range(1, r+1))
for i, j in combinations(range(1, r+1), 2):
    transposition = list(identity)
    transposition[i-1] = j
    transposition[j-1] = i
    transpositions.append(tuple(transposition))


temps = []
for comb in combinations_with_replacement(range(binomial(r, n) + 1), binomial(r, n)):
    aux = list([] for _ in range(n+1))
    for idx, i in enumerate(comb):
        try:
            aux[i].append(transpositions[idx])
        except IndexError:
            pass
    
    temps.append(aux)

pointed_answer = set()
for temp in temps:
    # add identities where needed
    temp = [[identity] if not x else x for x in temp]
    # get all permutations in each coordinate
    temp = [list(permutations(temp[i])) for i in range(len(temp))]
    # get all products
    for x in product(*temp):
        spx = tuple(reduce(act, y) for y in x)
        if complexity(spx) < 2:
            pointed_answer.add(spx)

print(((1,2,3), (1, 3, 2), (3, 1, 2)) in pointed_answer)
answer = set()
for spx, perm in product(pointed_answer, permutations(identity)):
    answer.add(tuple(act(s, perm) for s in spx))

simplices = set(x for x in answer if complexity(x) < 2)
simplices = tuple(spx for spx in simplices if is_nondegenerate(spx))

cell = []
for spx in simplices:
    in_cell = True
    for i, j in combinations(identity, 2):
#         print(phi(i, j, spx)[-1])
        if (phi(i, j, spx)[-1] == (2,1) and
            len([p for p, q in pairwise(phi(i, j, spx)) if p != q]) == 1):
            in_cell = False
    if in_cell:
        cell.append(spx)

print(len(simplices), len(cell))

print(((3, 2, 1), (2, 3, 1), (2, 1, 3)) in cell)

#### fails: we have an admissible term coming from id|(23)|(13)(23)

In [ ]:
xi = BarrattEccles_element({spx: 1 for spx in cell}, torsion=3)
print(xi.table_reduction())


In [ ]:
xi = BarrattEccles_element({spx: 1 for spx in cell}, torsion=3)
xi_s = Surjection_element({(3, 2, 3, 1, 3): 1, (3, 1, 3, 2, 3): 1, (3, 2, 1, 2, 3): 1})
y = Surjection_element({(3, 2, 1, 2, 3): 1})
# print(y)
z = BarrattEccles_element({((3, 2, 1), (2, 1, 3), (1, 2, 3)): 1, ((3, 2, 1), (3, 1, 2), (1, 2, 3)): 1})
wtt = BarrattEccles_element({((3, 2, 1), (2, 1, 3), (1, 2, 3)): 1})
print(z)
print(z.table_reduction())

In [ ]:
# finding twisted cycles
twisted_cycles = []

for coeffs in product(range(3), repeat=len(cell)):
    b = BarrattEccles_element({spx: coeff for spx, coeff in zip(cell, coeffs)},
                             torsion=3)
    if not symmetric_orbit_barratt_eccles(b.boundary(), sign_representation=True):
        twisted_cycles.append(b)
        
print('number of twisted cycles:', len(twisted_cycles))
        
# finding a basis of twisted cycles
cycles_basis = list()
zero = BarrattEccles_element(torsion=3)
aux = [zero]
for b in twisted_cycles:
    if b not in aux:
        cycles_basis.append(b)
        aux += [b + c for c in aux] + [2*b + c for c in aux]
        
print('number of basic cycles:', len(cycles_basis))

In [ ]:
# symmetric orbits are not working well

for b in twisted_cycles:
    print(symmetric_orbit_barratt_eccles(b.boundary(), sign_representation=True))
    print(sign_symmetric_orbit(b.table_reduction()))
    print('---')